# Create UniProt Mapping File

This notebook creates a file that maps Ensembl IDs to UniProt accession numbers. It uses all Ensembl IDs that are present in Agora input files (excluding druggability), queries UniProtKB for matching accession numbers, and writes the file to a tsv. This notebook uses UniprotKB-Swiss-Prot as its source, which ensures that all accessions returned have been reviewed and annotated by UniProt and are likely to be primary accessions only.

## Installation requirements

Install Python and agora-data-tools following the instructions in this repository's README. This notebook assumes it is being run from the same `pipenv` virtual environment as agora-data-tools. 

Then install the following packages using `pip`:
```
pip install unipressed
```

In [1]:
from unipressed import IdMappingClient
import time
import pandas as pd
import numpy as np
import agoradatatools.etl.utils as utils
import agoradatatools.etl.extract as extract

config_filename = "../../../../config.yaml"

## Get Ensembl IDs from data sets that will be processed by agora-data-tools

Loop through all data sets in the config file to get all Ensembl IDs used in every data set. NOTE: In the future, it would be simpler to just load the `gene_metadata` data set once druggability genes are removed from it, rather than looping through all of these files. 

In [2]:
config = utils._get_config(config_path=config_filename)
datasets = config["datasets"]

files = {}

for dataset in datasets:
    dataset_name = list(dataset.keys())[0]

    for entity in dataset[dataset_name]["files"]:
        entity_id = entity["id"]
        entity_format = entity["format"]
        entity_name = entity["name"]

        # Ignore json files, which are post-processed and not what we're interested in.
        # Also ignore "druggability" since we want to exclude druggability-only genes, and 
        # "gene_metadata" which includes druggability genes.
        if entity_format != "json" and entity_name not in ["druggability", "gene_metadata"]:
            files[entity_name] = (entity_id, entity_format)

# There are some duplicate synID's in this list but that doesn't really matter
files

{'genes_biodomains': ('syn44151254.5', 'csv'),
 'neuropath_regression_results': ('syn22017882.5', 'csv'),
 'proteomics': ('syn18689335.3', 'csv'),
 'proteomics_tmt': ('syn35221005.2', 'csv'),
 'proteomics_srm': ('syn52579640.4', 'csv'),
 'target_exp_validation_harmonized': ('syn24184512.9', 'csv'),
 'metabolomics': ('syn26064497.1', 'feather'),
 'igap': ('syn12514826.5', 'csv'),
 'eqtl': ('syn12514912.3', 'csv'),
 'diff_exp_data': ('syn27211942.1', 'tsv'),
 'target_list': ('syn12540368.47', 'csv'),
 'median_expression': ('syn27211878.2', 'csv'),
 'tep_adi_info': ('syn51942280.2', 'csv'),
 'team_info': ('syn12615624.18', 'csv'),
 'team_member_info': ('syn12615633.18', 'csv'),
 'overall_scores': ('syn25575156.13', 'table'),
 'networks': ('syn11685347.1', 'csv')}

### We should now have a list of all raw data files ingested. Get each one and create a list of IDs.

In [3]:
syn = utils._login_to_synapse(token=None)  # Assumes you have already logged in with a valid token

# The various column names used to store Ensembl IDs in the files
col_names = ["ENSG", "ensembl_gene_id", "GeneID", "ensembl_id"]
file_ensembl_list = []

for file in files.keys():
    df = extract.get_entity_as_df(syn_id=files[file][0], source=files[file][1], syn=syn)

    file_ensembl_ids = None

    for C in col_names:
        if C in df.columns:
            file_ensembl_ids = df[C]

    # networks file is a special case
    if file == "networks":
        file_ensembl_ids = pd.melt(
            df[["geneA_ensembl_gene_id", "geneB_ensembl_gene_id"]]
        )["value"]

    if file_ensembl_ids is not None:
        file_ensembl_list = file_ensembl_list + file_ensembl_ids.tolist()
        if "n/A" in file_ensembl_ids.tolist():
            print(file + " has an n/A Ensembl ID")
            file_ensembl_list.remove("n/A")
        if np.NaN in file_ensembl_ids.tolist():
            print(file + " has an NaN Ensembl ID")
    else:
        print("WARNING: no Ensembl ID column found for " + file + "!")


UPGRADE AVAILABLE

A more recent version of the Synapse Client (4.6.0) is available. Your version (4.0.0) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 4.6.0 release notes

https://python-docs.synapse.org/news/



Welcome, Jaclyn Beck!

INFO: 2024-11-15 11:43:36 | synapseclient_default | Welcome, Jaclyn Beck!

genes_biodomains has an NaN Ensembl ID


In [ ]:
file_ensembl_list = list(set(file_ensembl_list))

# NaNs will be floats, so this removes them. Using np.isnan() on strings throws an error.
ensembl_ids = [x for x in file_ensembl_list if isinstance(x, str)]

print(len(ensembl_ids))

35858


Query UniProt for accession numbers that match to Ensembl IDs. Using `UniProtKB-Swiss-Prot` ensures that all accession numbers returned have been reviewed and are highly likely to be primary accessions.

In [5]:
# Break the query into smaller chunks to avoid long jobs that could fail
batch_ind = range(0, len(ensembl_ids), 1000)
results = []

for B in batch_ind:
    end = min(len(ensembl_ids), B + 1000)
    print("Querying genes " + str(B + 1) + " - " + str(end))
    
    request = IdMappingClient.submit(
        source="Ensembl", dest="UniProtKB-Swiss-Prot", ids=ensembl_ids[B:end]
    )

    found = False
    while not found:
        time.sleep(2)
        
        status = request.get_status()
        if (status == "FINISHED"):
            results = results + list(request.each_result())
            found = True
        else:
            print("Waiting for response from UniProt...")

Querying genes 1 - 1000
Querying genes 1001 - 2000
Querying genes 2001 - 3000
Querying genes 3001 - 4000
Querying genes 4001 - 5000
Querying genes 5001 - 6000
Querying genes 6001 - 7000
Querying genes 7001 - 8000
Querying genes 8001 - 9000
Querying genes 9001 - 10000
Querying genes 10001 - 11000
Querying genes 11001 - 12000
Querying genes 12001 - 13000
Querying genes 13001 - 14000
Querying genes 14001 - 15000
Querying genes 15001 - 16000
Querying genes 16001 - 17000
Querying genes 17001 - 18000
Querying genes 18001 - 19000
Querying genes 19001 - 20000
Querying genes 20001 - 21000
Querying genes 21001 - 22000
Querying genes 22001 - 23000
Querying genes 23001 - 24000
Querying genes 24001 - 25000
Querying genes 25001 - 26000
Querying genes 26001 - 27000
Querying genes 27001 - 28000
Querying genes 28001 - 29000
Querying genes 29001 - 30000
Querying genes 30001 - 31000
Querying genes 31001 - 32000
Querying genes 32001 - 33000
Querying genes 33001 - 34000
Querying genes 34001 - 35000
Queryin

In [6]:
mapping = pd.DataFrame(results).rename(
    columns={"from": "RESOURCE_IDENTIFIER", "to": "UniProtKB_accession"}
)
mapping = mapping[["UniProtKB_accession", "RESOURCE_IDENTIFIER"]]
mapping

,UniProtKB_accession,RESOURCE_IDENTIFIER
0,A0A075B6I4,ENSG00000211642
1,Q13641,ENSG00000146242
2,Q6PCB7,ENSG00000130304
3,Q7Z591,ENSG00000106948
4,Q5SZD1,ENSG00000197261
...,...,...
18456,Q6ZUI0,ENSG00000188001
18457,O43747,ENSG00000166747
18458,Q9UBU2,ENSG00000155011
18459,Q86VY9,ENSG00000164484


In [7]:
mapping.to_csv(path_or_buf="../../output/ensg_to_uniprot_mapping.tsv", sep="\t", header=True, index=False)

# Extra information printouts

Total number of Ensembl IDs that match to a UniProt accession:

In [9]:
matches = len(mapping["RESOURCE_IDENTIFIER"].drop_duplicates())
total = len(ensembl_ids)
pct = round(matches * 100 / total, ndigits = 2)

print(f'{matches:.0f} of {total:.0f} ({pct:.2f}%) Ensembl IDs match to an accession')

18437 of 35858 (51.42%) Ensembl IDs match to an accession


Ensembl IDs that match to more than one UniProt accession:

In [10]:
dupes = mapping["RESOURCE_IDENTIFIER"].loc[mapping["RESOURCE_IDENTIFIER"].duplicated()].drop_duplicates()
print(f'{len(dupes):d} Ensembl IDs map to more than one UniProt accession')
mapping.loc[mapping["RESOURCE_IDENTIFIER"].isin(dupes)]

23 Ensembl IDs map to more than one UniProt accession


,UniProtKB_accession,RESOURCE_IDENTIFIER
538,P0CAP2,ENSG00000255529
539,Q6EEV4,ENSG00000255529
2499,O95467,ENSG00000087460
2500,P63092,ENSG00000087460
2501,Q5JWF2,ENSG00000087460
2846,P39880,ENSG00000257923
2847,Q13948,ENSG00000257923
2943,O96007,ENSG00000164172
2944,O96033,ENSG00000164172
4298,Q8NFQ8,ENSG00000169905


UniProt accessions that match to more than one Ensembl ID:

In [11]:
dupes2 = mapping["UniProtKB_accession"].loc[mapping["UniProtKB_accession"].duplicated()].drop_duplicates()
print(f'{len(dupes2):d} UniProt accessions map to more than one Ensembl ID')
mapping.loc[mapping["UniProtKB_accession"].isin(dupes2)]

28 UniProt accessions map to more than one Ensembl ID


,UniProtKB_accession,RESOURCE_IDENTIFIER
498,Q08493,ENSG00000285188
664,Q5JQF8,ENSG00000184388
845,P62805,ENSG00000197061
1474,Q71DI3,ENSG00000203852
1553,P0C0S8,ENSG00000196747
...,...,...
17564,Q08493,ENSG00000105650
18069,P01562,ENSG00000197919
18161,P62805,ENSG00000278705
18253,P62807,ENSG00000277224
